In [1]:
import pandas as pd
import numpy as np

#visualization
import seaborn as sns
import matplotlib.pyplot as plt


#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Data Scalling
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler,MaxAbsScaler


##
from sklearn.preprocessing import Normalizer,Binarizer




#Data encoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Data Splitting
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#Regressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge, BayesianRidge, ElasticNet, HuberRegressor,LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
##Polynomial
from sklearn.preprocessing import PolynomialFeatures as PF


##Pipeline
from sklearn.pipeline import make_pipeline

#statistic
from scipy import stats


import warnings 
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("zomato.csv")

In [3]:
df = df.dropna(axis=0,subset=['rate'])
df = df.dropna(axis=0,subset=['cuisines'])
df.columns = ['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'cost_two_person', 'reviews_list', 'menu_item',
       'service_type', 'registered_city']
df['rate'].replace('NEW','0', inplace=True)
df['rate'].replace("-",np.nan,inplace=True)
df['rate'] = df['rate'].astype('str')
df['rate'] = df['rate'].apply(lambda x: x.split("/")[0])
#sebelah paling kanan terdapat spasi dihapus
df['rate'] = df['rate'].str.rstrip()
df['rate'] = df['rate'].astype('float')

In [4]:
df.head(1)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost_two_person,reviews_list,menu_item,service_type,registered_city
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari


In [5]:
df['cost_two_person'] = df[~(df['cost_two_person'].isna())]['cost_two_person'].apply(lambda x: x.split(","))
df['cost_two_person'] = df[~(df['cost_two_person'].isna())]['cost_two_person'].apply(lambda x: "".join(x))
df['cost_two_person'] = df['cost_two_person'].astype("float")
df.dropna(subset=['cost_two_person'],inplace=True)
cols = ['url','address','phone','name','reviews_list','menu_item','registered_city','dish_liked', 'cuisines', 'rest_type']
df = df.drop(cols,axis=1)

In [6]:
df.head()

,online_order,book_table,rate,votes,location,cost_two_person,service_type
0,Yes,Yes,4.1,775,Banashankari,800.0,Buffet
1,Yes,No,4.1,787,Banashankari,800.0,Buffet
2,Yes,No,3.8,918,Banashankari,800.0,Buffet
3,No,No,3.7,88,Banashankari,300.0,Buffet
4,No,No,3.8,166,Basavanagudi,600.0,Buffet


In [7]:
df.groupby('service_type').mean()['rate'].sort_values()

service_type
Delivery              3.457655
Dine-out              3.494039
Desserts              3.595896
Cafes                 3.682064
Pubs and bars         3.851970
Buffet                3.898614
Drinks & nightlife    3.911556
Name: rate, dtype: float64

In [8]:
temp_dict2 = {}
for i,v in enumerate(df.groupby('location').mean()['rate'].sort_values().items()):
    temp_dict2[v[0]] = i + 1
temp_dict2

{'West Bangalore': 1,
 'North Bangalore': 2,
 'Hebbal': 3,
 'Bommanahalli': 4,
 'Electronic City': 5,
 'Hennur': 6,
 'Thippasandra': 7,
 'Rammurthy Nagar': 8,
 'Commercial Street': 9,
 'Ejipura': 10,
 'Old Madras Road': 11,
 'Peenya': 12,
 'CV Raman Nagar': 13,
 'KR Puram': 14,
 'East Bangalore': 15,
 'Vijay Nagar': 16,
 'Bannerghatta Road': 17,
 'Wilson Garden': 18,
 'HBR Layout': 19,
 'Koramangala 1st Block': 20,
 'RT Nagar': 21,
 'Majestic': 22,
 'BTM': 23,
 'Varthur Main Road, Whitefield': 24,
 'Bellandur': 25,
 'ITPL Main Road, Whitefield': 26,
 'Kumaraswamy Layout': 27,
 'Shanti Nagar': 28,
 'Nagawara': 29,
 'Banaswadi': 30,
 'Sanjay Nagar': 31,
 'Banashankari': 32,
 'Brookefield': 33,
 'Whitefield': 34,
 'Marathahalli': 35,
 'Nagarbhavi': 36,
 'Domlur': 37,
 'Old Airport Road': 38,
 'JP Nagar': 39,
 'South Bangalore': 40,
 'Rajajinagar': 41,
 'Magadi Road': 42,
 'Basaveshwara Nagar': 43,
 'Jeevan Bhima Nagar': 44,
 'City Market': 45,
 'Mysore Road': 46,
 'Kaggadasapura': 47,
 'S

In [9]:
temp_dict = {'Delivery': 1, 'Dine-out': 2, 'Desserts': 3, 'Cafes': 4, 
             'Pubs and bars': 5, 'Buffet': 6, 'Drinks & nightlife': 7}

df['service_type'] = df['service_type'].map(temp_dict)

In [10]:
temp_dict1 = {'No': 0, 'Yes': 1}

df['online_order'] = df['online_order'].map(temp_dict1)
df['book_table'] = df['book_table'].map(temp_dict1)

In [11]:
df['location'] = df['location'].map(temp_dict2)

In [12]:
# df['online_order'] = LabelEncoder().fit_transform(df['online_order'])
# df['book_table'] = LabelEncoder().fit_transform(df['book_table'])
# df['location'] = LabelEncoder().fit_transform(df['location'])
# df['service_type'] = LabelEncoder().fit_transform(df['service_type'])

In [13]:
df.dropna(inplace=True)
df.head()

,online_order,book_table,rate,votes,location,cost_two_person,service_type
0,1,1,4.1,775,32,800.0,6
1,1,0,4.1,787,32,800.0,6
2,1,0,3.8,918,32,800.0,6
3,0,0,3.7,88,32,300.0,6
4,0,0,3.8,166,60,600.0,6


In [19]:
print(temp_dict)
print(temp_dict1)
print(temp_dict2)



{'Delivery': 1, 'Dine-out': 2, 'Desserts': 3, 'Cafes': 4, 'Pubs and bars': 5, 'Buffet': 6, 'Drinks & nightlife': 7}
{'No': 0, 'Yes': 1}
{'West Bangalore': 1, 'North Bangalore': 2, 'Hebbal': 3, 'Bommanahalli': 4, 'Electronic City': 5, 'Hennur': 6, 'Thippasandra': 7, 'Rammurthy Nagar': 8, 'Commercial Street': 9, 'Ejipura': 10, 'Old Madras Road': 11, 'Peenya': 12, 'CV Raman Nagar': 13, 'KR Puram': 14, 'East Bangalore': 15, 'Vijay Nagar': 16, 'Bannerghatta Road': 17, 'Wilson Garden': 18, 'HBR Layout': 19, 'Koramangala 1st Block': 20, 'RT Nagar': 21, 'Majestic': 22, 'BTM': 23, 'Varthur Main Road, Whitefield': 24, 'Bellandur': 25, 'ITPL Main Road, Whitefield': 26, 'Kumaraswamy Layout': 27, 'Shanti Nagar': 28, 'Nagawara': 29, 'Banaswadi': 30, 'Sanjay Nagar': 31, 'Banashankari': 32, 'Brookefield': 33, 'Whitefield': 34, 'Marathahalli': 35, 'Nagarbhavi': 36, 'Domlur': 37, 'Old Airport Road': 38, 'JP Nagar': 39, 'South Bangalore': 40, 'Rajajinagar': 41, 'Magadi Road': 42, 'Basaveshwara Nagar': 43

---

In [17]:
import joblib

In [18]:
x = df.drop(['rate'],axis=1)
y = df['rate']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

rfr = RandomForestRegressor(random_state=42)
lnr = LinearRegression()
dct = DecisionTreeRegressor(random_state=42)

model = rfr.fit(x_train,y_train)
lnr.fit(x_train,y_train)
dct.fit(x_train,y_train)

rfr_y_train_predict = rfr.predict(x_train)
rfr_y_test_predict = rfr.predict(x_test)
lnr_y_train_predict = lnr.predict(x_train)
lnr_y_test_predict = lnr.predict(x_test)
dct_y_train_predict = dct.predict(x_train)
dct_y_test_predict = dct.predict(x_test)

joblib.dump(model,"model_final_rfr")

RandomForest_Result = pd.DataFrame({'Dataset': ['Random Forest Regressor'], 
              'CV error': [ (cross_val_score(rfr, x, y, scoring='neg_root_mean_squared_error',cv=5).mean())],
              'CV std': [ (cross_val_score(rfr, x, y, scoring='neg_root_mean_squared_error',cv=5).std())],
              'Training Error' : [np.sqrt(mean_squared_error(y_train, rfr_y_train_predict))],
              'Test Error' : [np.sqrt(mean_squared_error(y_test, rfr_y_test_predict))],
              'Training_R2_Score' : [r2_score(y_train, rfr_y_train_predict)],
              'Test_R2_Score' : [r2_score(y_test, rfr_y_test_predict)]
             }).set_index('Dataset')
RandomForest_Result

,CV error,CV std,Training Error,Test Error,Training_R2_Score,Test_R2_Score
Dataset,,,,,,
Random Forest Regressor,-0.212586,0.037812,0.089611,0.1905,0.990565,0.954979
